# Project : CabEdge

## This file lists all the functions created for the project

In [3]:
import pandas as pd

#### Dividing the pickup and dropoff timestamps in date and time

In [1]:
def time_to_date(df):
    date_list = []
    date_list = df.split(' ')
    if(len(date_list) == 2):
        return date_list[0]
    else:
        print(date_list)
        return np.nan

In [2]:
def time_to_time(df):
    date_list = []
    date_list = df.split(' ')
    if(len(date_list) == 2):
        return date_list[1]
    else:
        print(date_list)
        return np.nan

#### Determining the 'Hour' from the timestamp

In [3]:
def time_to_hour(df):
    time_str=df
    time=[]
    time=time_str.split(':')
    return time[0]

#### Determining the 'Day' from the timestamp

In [4]:
def date_to_day(df):
    date_str=df
    date=[]
    date=date_str.split('-')
    return date[2]


#### Determining the 'Month' from the timestamp

In [5]:
def date_to_month(df):
    month_str=df
    month=[]
    month=month_str.split('-')
    return month[1]

#### Determining the 'Year' from the timestamp

In [6]:
def date_to_year(df):
    year_str=df
    year=[]
    year=df.split('-')
    return year[0]

#### Was the day weekday or Weekend???

In [7]:
"""Was the day weekday or Weekend???"""
"""Weekday : wkd"""
"""Weekend : Wkd"""
"""Monday=1,Tuesday=2,Wednesday=3,Thursday=4,Friday=5,Saturday=6 Sunday=7"""

def weekday_or_weekend(df):
    wk = df.weekday()+1
    return wk



#### Lets create the target with time of the day and drop of zone 

In [8]:
"""1. Lets create the target with time of the day and drop of zone """

def target_hotzone(time,day,drop_zone,df):
    filtered_df = df[(df['type_of_day'] == day) & (df['hour'] == time)]
#     print(filtered_df['PULocationID'].value_counts().index)
    i=0
    target=[]
    while(i<3):
        try:
            target.append(filtered_df['PULocationID'].value_counts().index[i])
            i +=1
        except:
            return tuple(target)
    return tuple(target)

#### Creating a function to convert from unix time to utc time

In [9]:
import time

In [10]:
def unix_to_utc(df):
    time_str = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(df))
    return time_str

## Implementation of Logic to determine the Zone the Driver to Drive to

#### Creating Function to determine demand and supply

In [11]:
def demand_supply(pick__time_series,drop_time_series,month,day,hour,zone):
    
    
    """Create the demand % for origin"""
    nb_pick = pick__time_series[(pick__time_series['Month'] == month) & (pick__time_series['Day'] == day) & (pick__time_series['hour'] == hour) & (pick__time_series['PULocationID'] == zone)]['Count'].values[0]
    nb_drop = drop_time_series[(drop_time_series['drop_off_Month'] == month) & (drop_time_series['drop_off_Day'] == day) & (drop_time_series['drop_off_Hour'] == hour) & (drop_time_series['DOLocationID'] == zone)]['Count'].values[0]
    
    """ If demand_origin is greater than 1 then there is demand"""
    demand_origin = float((nb_pick/nb_drop))
    print('Current Zone: ',zone, ': ',demand_origin)
    """Create the demand % at all locations"""
    zone_demand_list = []
    
    for zone_id in range(1,264):
        try:
#             print(pick__time_series[(pick__time_series['Month'] == month) & (pick__time_series['Day'] == day) & (pick__time_series['hour'] == hour) & (pick__time_series['PULocationID'] == zone_id)]['Count'].values[0])
            nb_pick = pick__time_series[(pick__time_series['Month'] == month) & (pick__time_series['Day'] == day) & (pick__time_series['hour'] == hour) & (pick__time_series['PULocationID'] == zone_id)]['Count'].values[0]
            nb_drop = drop_time_series[(drop_time_series['drop_off_Month'] == month) & (drop_time_series['drop_off_Day'] == day) & (drop_time_series['drop_off_Hour'] == hour) & (drop_time_series['DOLocationID'] == zone_id)]['Count'].values[0]
            demand_zone = float((nb_pick/nb_drop)) #Ratio of pick up and supply

#             print('Demand Origin :',demand_origin)
#             print('Demand Zone: ',demand_zone)
            #If any zone's demand % is greater than 1, and 20% higher than demand % of current location then consider it
          
            threshold = 1.2 * demand_origin

#             print(threshold)
#           If demand_zone is greater than threshold and is +ve
            if((demand_zone > threshold) & (demand_zone > 1)):
                print('zone :',zone_id,', ', demand_zone)
                zone_demand_list.append(zone_id)
#                 print(zone_demand_list)
        except :
            pass
        
    return zone_demand_list

In [6]:
def model_demand_supply(pick_zone_dict,drop_zone_dict,current_zone):
    
    
    """Create the demand % for origin"""
    """ If demand_origin is greater than 1 then there is demand"""
    try:
        demand_origin = pick_zone_dict[current_zone]/drop_zone_dict[current_zone]
        print('Current Zone: ',current_zone, ': ',demand_origin)
    except:
        demand_origin = 1
    
    """Create the demand % at 10 locations"""
    zone_demand_list = []
    
    for zone_id in pick_zone_dict.keys():
        try:
            #Ratio of pick up and supply
            demand_zone = float((pick_zone_dict[zone_id]/drop_zone_dict[zone_id])) 

            #If any zone's demand % is greater than 1, and 20% higher than demand % of current location then consider it
          
            threshold = 1.2 * demand_origin

#             print(threshold)
#           If demand_zone is greater than threshold and is +ve
            if((demand_zone > threshold) & (demand_zone > 1)):
                print('demand zone :',zone_id,', ', demand_zone)
                zone_demand_list.append(zone_id)
#                 print(zone_demand_list)
        except :
            pass
        
    return zone_demand_list

#### So now logic for finding out the best zone amongst the filtered list of zones

In [2]:
def find_best_zone_distance(zone_square_dist_df,current_zone,zone_demand_list):

    best_zone_distance_list = []
    
    if len(zone_demand_list) > 0:
        for zone in zone_demand_list:
            dist = zone_square_dist_df.loc[current_zone][zone]
            best_zone_distance_list.append(dist)
    else:
        zone_demand_list.append(current_zone)
        best_zone_distance_list.append(0)
        print(zone_demand_list,best_zone_distance_list)
        
    import pandas as pd
    best_zone_distance = pd.DataFrame()
    
    best_zone_distance['zone'] = zone_demand_list
    
    best_zone_distance['Distance'] = best_zone_distance_list
    
    best_zone = best_zone_distance.sort_values(by='Distance').iloc[0,0]
    
    print(best_zone)
    
    return best_zone
    

#### Print the name of Best Zone

In [4]:
def best_zone_func(best_zone):
    
    zone_df = pd.read_csv('/home/jupyter/capstone/database/zone_df.csv')
    borough = zone_df[zone_df['LocationID'] == best_zone]['Borough'].values[0]
    zone = zone_df[zone_df['LocationID'] == best_zone]['Zone'].values[0]
    
    return borough, zone

### Logic to determine the best zone based on Median prices

In [15]:
def find_best_zone_fare(fare_time_series,current_zone,zone_demand_list,month,day,hour):

    best_zone_fare_list = []
    new_zone_list =[]
    
    new_fare_time_series = fare_time_series[(fare_time_series['Month'] == str(month)) & (fare_time_series['Day'] == str(day)) & (fare_time_series['hour']== str(hour))]
#     print(new_fare_time_series)
    current_zone_median_fare = new_fare_time_series[new_fare_time_series['PULocationID'] == current_zone]['Median Fare amount'].values[0]
    
    print(current_zone_median_fare)
    
    for zone in zone_demand_list:
        median_fare = new_fare_time_series[new_fare_time_series['PULocationID'] == zone]['Median Fare amount'].values[0]
        
        #If median fare of the zone is greater than 20% of the current zone
        
        if(median_fare > 1.2 * current_zone_median_fare):
            best_zone_fare_list.append(median_fare)
            new_zone_list.append(zone)
    
    
    if(len(best_zone_fare_list) > 0):
        best_zone_fare = pd.DataFrame()

        best_zone_fare['zone'] = new_zone_list
        best_zone_fare['Median Fare'] = best_zone_fare_list

        best_zone = best_zone_fare.sort_values(by='Median Fare',ascending=False).iloc[0,0]
        best_median_fare = best_zone_fare.sort_values(by='Median Fare',ascending=False).iloc[0,1]

        print(best_zone,': ',best_median_fare)
    else:
        best_zone = current_zone
    
    return best_zone